# Tuple Luminosity Percentages

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
# you need to install BigFile
from bigfile import BigFile
import glob,os,struct
# from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from scipy.interpolate import interp1d
from colossus.cosmology import cosmology

import seaborn as sns

import plotly.express as px
import pandas as pd

sns.set()
sns.set_palette("Set2")
sns.set_style('ticks',{'ytick.direction':'in','xtick.direction':'in'})

cmap = plt.get_cmap("Set2")
sns.set_context("paper", font_scale=1.7,rc={"axes.linewidth": 1.3,"lines.linewidth": 2.5,"patch.linewidth": 2.2})
from matplotlib import rcParams as rc
import pickle
import warnings

import import_ipynb
import Tuples_Information as tuples_z2_z3

importing Jupyter notebook from Tuples Information.ipynb


In [2]:
warnings.filterwarnings(action='ignore', message='Mean of empty slice')
warnings.filterwarnings(action='ignore', message='invalid value encountered in double_scalars')

In [3]:
# some constants and unit conversions
msun_mks = 1.989e30
pc_mks = 3.086e16
grav_mks = 6.67e-11
km_mks = 1e3
yr_mks = 3.154e+7
c_mks = 3e8

Mpc_to_m = 3.086e+22
m_to_Mpc = 1./Mpc_to_m
s_to_year = 3.17098e-8
c_Mpc_yr = c_mks*m_to_Mpc/s_to_year

In [4]:
# These arrays are necessary for locating a BH in the detials file
path = '/hildafs/datasets/Asterix/BH_details_dict/Read-Blackhole-Detail'
detail = BigFile(path)
AllIDs = detail.open('BHID')[:]
Index = detail.open('Index')[:]

In [5]:
c_mks = 3e8
msun_mks = 2e30
s_to_year = 3.17098e-8
year_to_s = 1./s_to_year
lsun_ergs = 3.9e33
mdot_msun_yr = 1e10/980/1e6
def calc_lx(mdot):
    """
    input: mdot in Msun/yr
    output: Lx in ergs
    """
    lbol = 0.1*mdot*msun_mks/year_to_s*c_mks**2
    lbol_lsun = lbol/3.9e26
    k = 10.83*(lbol_lsun/1e10)**0.28 \
        + 6.08*(lbol_lsun/1e10)**(-0.02)
    return lbol/k*1e7

def calc_lbol(mdot):
    """
    input: mdot in Msun/yr
    output: Lx in ergs
    """
    lbol = 0.1*mdot*msun_mks/year_to_s*c_mks**2
    lbol_ergs = lbol*1e7
    return lbol_ergs
def density_to_msunpc3(a): # density in mass/len^3 unit
    mass_msun = 1e10/hh
    length_pc = a*1000/hh
    rho = mass_msun/(length_pc)**3
    return rho

In [6]:
def get_bh_history(bhid):
    idx = (AllIDs==bhid).nonzero()[0][0]
    chunk = Index[idx]
    # print('File number of the target BHs:',chunk)
    # now load bh data
    outdir = '/hildafs/datasets/Asterix/BH_details_dict/'
    save = outdir+'BlackholeDetails-%04d'%chunk
    with open(save, 'rb') as f:
        data = pickle.load(f)
        f.close()
    bh = data[bhid]
    return bh

In [7]:
def trip_perc(bh1,bh2,bh3,zstart,zend,bins):
    mask1 = bh1['z'] <= zstart
    mask1 &= bh1['z'] >= zend
    mask2 = bh2['z'] <= zstart
    mask2 &= bh2['z'] >= zend
    mask3 = bh3['z'] <= zstart
    mask3 &= bh3['z'] >= zend
    
    bh1 = bh1[mask1]
    bh2 = bh2[mask2]
    bh3 = bh3[mask3]
    
    t_start = z_to_t(zstart)
    t_end = z_to_t(zend)
    # experiment with the number of samples
    tt = np.linspace(t_start,t_end,bins)
    zz = t_to_z(tt)
    
    dt = tt[1] - tt[0]
    dz = zz[0] - zz[1]
    
    lum_temp = []
    
    for i in range(1,bins):
        temp_mask1 = bh1['z'] <= t_to_z(tt[i - 1])
        temp_mask1 &= bh1['z'] >= t_to_z(tt[i])
        temp_mask2 = bh2['z'] <= t_to_z(tt[i - 1])
        temp_mask2 &= bh2['z'] >= t_to_z(tt[i])
        temp_mask3 = bh3['z'] <= t_to_z(tt[i - 1])
        temp_mask3 &= bh3['z'] >= t_to_z(tt[i])
    
        temp_bh1 = bh1[temp_mask1]
        temp_bh2 = bh2[temp_mask2]
        temp_bh3 = bh3[temp_mask3]
        
        avg1 = calc_lx(np.mean(temp_bh1['Mdot'])*mdot_msun_yr)
        avg2 = calc_lx(np.mean(temp_bh2['Mdot'])*mdot_msun_yr)
        avg3 = calc_lx(np.mean(temp_bh3['Mdot'])*mdot_msun_yr)
        
        lum_temp.append((avg1,avg2,avg3,t_to_z(tt[i - 1]),tt[i - 1],(tt[i] - tt[i - 1])))

    lum_info = np.array(lum_temp,
                        dtype=[('BH1LumX', '<f8'), ('BH2LumX', '<f8'), ('BH3LumX', '<f8'),
                               ('z', '<f8'), ('t', '<f8'), ('dt', '<f8')])
    
    lum_mask1 = lum_info['BH1LumX'] >= 10**42
    lum_mask1 |= lum_info['BH2LumX'] >= 10**42
    lum_mask1 |= lum_info['BH3LumX'] >= 10**42
    
    lum_mask2 = [((el['BH1LumX'] >= 10**42 and el['BH2LumX'] >= 10**42)
                 or (el['BH1LumX'] >= 10**42 and el['BH3LumX'] >= 10**42)
                 or (el['BH2LumX'] >= 10**42 and el['BH3LumX'] >= 10**42)) for el in lum_info]
    
    lum_mask3 = lum_info['BH1LumX'] >= 10**42
    lum_mask3 &= lum_info['BH2LumX'] >= 10**42
    lum_mask3 &= lum_info['BH3LumX'] >= 10**42
    
    time1 = np.sum(lum_info[lum_mask1]['dt'])
    time2 = np.sum(lum_info[lum_mask2]['dt'])
    time3 = np.sum(lum_info[lum_mask3]['dt'])
    
    totaltime = np.sum(lum_info['dt'])
    
    perc1 = 100 * time1 / totaltime
    perc2 = 100 * time2 / totaltime
    perc3 = 100 * time3 / totaltime
    
    # print("Percent of 1 BH's L_x >= 10**42:",perc1)
    # print("Percent of 2 BH's L_x >= 10**42:",perc2)
    # print("Percent of 3 BH's L_x >= 10**42:",perc3)
    
    return(perc1,perc2,perc3,time1,time2,time3,totaltime)

In [8]:
def quad_perc(bh1,bh2,bh3,bh4,zstart,zend,bins):
    mask1 = bh1['z'] <= zstart
    mask1 &= bh1['z'] >= zend
    mask2 = bh2['z'] <= zstart
    mask2 &= bh2['z'] >= zend
    mask3 = bh3['z'] <= zstart
    mask3 &= bh3['z'] >= zend
    mask4 = bh4['z'] <= zstart
    mask4 &= bh4['z'] >= zend
    
    bh1 = bh1[mask1]
    bh2 = bh2[mask2]
    bh3 = bh3[mask3]
    bh4 = bh4[mask4]
    
    t_start = z_to_t(zstart)
    t_end = z_to_t(zend)
    # experiment with the number of samples
    tt = np.linspace(t_start,t_end,bins)
    zz = t_to_z(tt)
    
    dt = tt[1] - tt[0]
    dz = zz[0] - zz[1]
    
    lum_temp = []
    
    for i in range(1,bins):
        temp_mask1 = bh1['z'] <= t_to_z(tt[i - 1])
        temp_mask1 &= bh1['z'] >= t_to_z(tt[i])
        temp_mask2 = bh2['z'] <= t_to_z(tt[i - 1])
        temp_mask2 &= bh2['z'] >= t_to_z(tt[i])
        temp_mask3 = bh3['z'] <= t_to_z(tt[i - 1])
        temp_mask3 &= bh3['z'] >= t_to_z(tt[i])
        temp_mask4 = bh4['z'] <= t_to_z(tt[i - 1])
        temp_mask4 &= bh4['z'] >= t_to_z(tt[i])
    
        temp_bh1 = bh1[temp_mask1]
        temp_bh2 = bh2[temp_mask2]
        temp_bh3 = bh3[temp_mask3]
        temp_bh4 = bh4[temp_mask4]
        
        avg1 = calc_lx(np.mean(temp_bh1['Mdot'])*mdot_msun_yr)
        avg2 = calc_lx(np.mean(temp_bh2['Mdot'])*mdot_msun_yr)
        avg3 = calc_lx(np.mean(temp_bh3['Mdot'])*mdot_msun_yr)
        avg4 = calc_lx(np.mean(temp_bh4['Mdot'])*mdot_msun_yr)
        
        lum_temp.append((avg1,avg2,avg3,avg4,t_to_z(tt[i - 1]),tt[i - 1],(tt[i] - tt[i - 1])))

    lum_info = np.array(lum_temp,
                        dtype=[('BH1LumX', '<f8'), ('BH2LumX', '<f8'), ('BH3LumX', '<f8'), ('BH4LumX', '<f8'),
                               ('z', '<f8'), ('t', '<f8'), ('dt', '<f8')])
    
    lum_mask1 = lum_info['BH1LumX'] >= 10**42
    lum_mask1 |= lum_info['BH2LumX'] >= 10**42
    lum_mask1 |= lum_info['BH3LumX'] >= 10**42
    lum_mask1 |= lum_info['BH4LumX'] >= 10**42
    
    lum_mask2 = [((el['BH1LumX'] >= 10**42 and el['BH2LumX'] >= 10**42)
                  or (el['BH1LumX'] >= 10**42 and el['BH3LumX'] >= 10**42)
                  or (el['BH1LumX'] >= 10**42 and el['BH4LumX'] >= 10**42)
                  or (el['BH2LumX'] >= 10**42 and el['BH3LumX'] >= 10**42)
                  or (el['BH2LumX'] >= 10**42 and el['BH4LumX'] >= 10**42)
                  or (el['BH3LumX'] >= 10**42 and el['BH4LumX'] >= 10**42)) for el in lum_info]
    
    lum_mask3 = [((el['BH1LumX'] >= 10**42 and el['BH2LumX'] >= 10**42 and el['BH3LumX'] >= 10**42)
                  or (el['BH1LumX'] >= 10**42 and el['BH2LumX'] >= 10**42 and el['BH4LumX'] >= 10**42)
                  or (el['BH1LumX'] >= 10**42 and el['BH3LumX'] >= 10**42 and el['BH4LumX'] >= 10**42)
                  or (el['BH2LumX'] >= 10**42 and el['BH3LumX'] >= 10**42 and el['BH4LumX'] >= 10**42)) for el in lum_info]
    
    lum_mask4 = lum_info['BH1LumX'] >= 10**42
    lum_mask4 &= lum_info['BH2LumX'] >= 10**42
    lum_mask4 &= lum_info['BH3LumX'] >= 10**42
    lum_mask4 &= lum_info['BH4LumX'] >= 10**42
    
    time1 = np.sum(lum_info[lum_mask1]['dt'])
    time2 = np.sum(lum_info[lum_mask2]['dt'])
    time3 = np.sum(lum_info[lum_mask3]['dt'])
    time4 = np.sum(lum_info[lum_mask4]['dt'])
    
    totaltime = np.sum(lum_info['dt'])
    
    perc1 = 100 * time1 / totaltime
    perc2 = 100 * time2 / totaltime
    perc3 = 100 * time3 / totaltime
    perc4 = 100 * time4 / totaltime
    
    # print("Percent of 1 BH's L_x >= 10**42:",perc1)
    # print("Percent of 2 BH's L_x >= 10**42:",perc2)
    # print("Percent of 3 BH's L_x >= 10**42:",perc3)
    # print("Percent of 4 BH's L_x >= 10**42:",perc4)
    return (perc1,perc2,perc3,perc4,time1,time2,time3,time4,totaltime)

In [9]:
triples_z2_dr30 = tuples_z2_z3.triples_z2_dr30
triples_z3_dr30 = tuples_z2_z3.triples_z3_dr30
triples_z2_dr200 = tuples_z2_z3.triples_z2_dr200
triples_z3_dr200 = tuples_z2_z3.triples_z3_dr200

quadruples_z2_dr30 = tuples_z2_z3.quadruples_z2_dr30
quadruples_z3_dr30 = tuples_z2_z3.quadruples_z3_dr30
quadruples_z2_dr200 = tuples_z2_z3.quadruples_z2_dr200
quadruples_z3_dr200 = tuples_z2_z3.quadruples_z3_dr200

## Calculating LumX Percentages for Triples

### z = 3

In [10]:
pig = BigFile('/hildafs/datasets/Asterix/PIG_files/PIG_214')
battr = pig["Header"].attrs
scale_fac = battr["Time"][0]
redshift = 1./battr["Time"][0] - 1
Lbox = battr['BoxSize']
hh = battr['HubbleParam']
om0 = battr['Omega0']
omb = battr['OmegaBaryon']
oml = battr['OmegaLambda']
Nfof = battr['NumFOFGroupsTotal']
sigma8 = 0.82
    
BH_IDs = pig.open('5/ID')[:]
lbt = pig.open('FOFGroups/LengthByType')[:]
OffsetByType = np.cumsum(lbt,axis=0)
a1 = np.array([[0,0,0,0,0,0]],dtype=np.uint64)
OffsetByType = np.append(a1,OffsetByType,axis=0)
bhoff = OffsetByType[:,5]

# set-up cosmology
params = {'flat': True, 'H0': 100*hh, 'Om0': om0, 'Ob0': omb, 'sigma8': sigma8, 'ns': 0.96}
cosmo = cosmology.setCosmology('myCosmo', params)

# conversion between time and redshift
z_arr = np.linspace(0,10,500)
time = cosmo.age(z_arr) # Gyr
def z_to_t(x):
    return interp1d(z_arr, time,fill_value='extrapolate')(x)
def t_to_z(x):
    return interp1d(time, z_arr,fill_value='extrapolate')(x) 

In [11]:
triples_z3_dr30_perc = []

for i in range (0, len(triples_z3_dr30)):
    
    id1,id2,id3 = triples_z3_dr30[i][0],triples_z3_dr30[i][1],triples_z3_dr30[i][2]

    bh1 = get_bh_history(id1)
    bh2 = get_bh_history(id2)
    bh3 = get_bh_history(id3)
    
    idx = i + 1
    perc1,perc2,perc3,time1,time2,time3,totaltime = trip_perc(bh1,bh2,bh3,zstart=4.0,zend=redshift,bins=500)
    id1,id2,id3 = triples_z3_dr30[i][0],triples_z3_dr30[i][1],triples_z3_dr30[i][2]
    
    triples_z3_dr30_perc.append((idx,id1,id2,id3,perc1,perc2,perc3,time1,time2,time3,totaltime))

# np.savetxt("triples_z3_dr30 BH Percentages 01 - 36.csv",triples_z3_dr30_z3_dr30_perc,delimiter=", ",fmt='% s')

trips_z3_dr30_perc = np.array(triples_z3_dr30_perc,dtype=[('TripIDX', '<f8'), ('BH1ID', '<f8'), ('BH2ID', '<f8'),
                                                          ('BH3ID', '<f8'), ('Perc1', '<f8'), ('Perc2', '<f8'),
                                                          ('Perc3', '<f8'), ('Time1', '<f8'), ('Time2', '<f8'),
                                                          ('Time3', '<f8'), ('TotalTime', '<f8')])

In [12]:
triples_z3_dr200_perc = []

for i in range (0, len(triples_z3_dr200)):
    
    id1,id2,id3 = triples_z3_dr200[i][0],triples_z3_dr200[i][1],triples_z3_dr200[i][2]

    bh1 = get_bh_history(id1)
    bh2 = get_bh_history(id2)
    bh3 = get_bh_history(id3)
    
    idx = i + 1
    perc1,perc2,perc3,time1,time2,time3,totaltime = trip_perc(bh1,bh2,bh3,zstart=4.0,zend=redshift,bins=500)
    id1,id2,id3 = triples_z3_dr200[i][0],triples_z3_dr200[i][1],triples_z3_dr200[i][2]
    
    triples_z3_dr200_perc.append((idx,id1,id2,id3,perc1,perc2,perc3,time1,time2,time3,totaltime))

# np.savetxt("triples_z3_dr200 BH Percentages 01 - 36.csv",triples_z3_dr200_z3_dr200_perc,delimiter=", ",fmt='% s')

trips_z3_dr200_perc = np.array(triples_z3_dr200_perc,dtype=[('TripIDX', '<f8'), ('BH1ID', '<f8'), ('BH2ID', '<f8'),
                                                          ('BH3ID', '<f8'), ('Perc1', '<f8'), ('Perc2', '<f8'),
                                                          ('Perc3', '<f8'), ('Time1', '<f8'), ('Time2', '<f8'),
                                                          ('Time3', '<f8'), ('TotalTime', '<f8')])

In [13]:
trips_z3_perc = np.concatenate((trips_z3_dr30_perc,trips_z3_dr200_perc))

In [14]:
trips_z3_dr30_perc_df = pd.DataFrame(trips_z3_dr30_perc)
trips_z3_dr200_perc_df = pd.DataFrame(trips_z3_dr200_perc)
trips_z3_perc_df = pd.DataFrame(trips_z3_perc)

#### Plotting LumX Percentages for Triples at z = 3

In [20]:
fig = px.density_heatmap(trips_z3_dr30_perc_df, x="Perc2", y="Perc3",text_auto=True,nbinsx=5,nbinsy=5,
                         labels={
                           "Perc2": "2 AGN [%]",
                           "Perc3": "3 AGN [%]"  
                         },
                         marginal_x="histogram",marginal_y="histogram")

fig.show()

In [19]:
fig = px.density_heatmap(trips_z3_dr200_perc_df, x="Perc2", y="Perc3",text_auto=True,nbinsx=5,nbinsy=5,
                         labels={
                           "Perc2": "2 AGN [%]",
                           "Perc3": "3 AGN [%]"  
                         },
                         marginal_x="histogram",marginal_y="histogram")

fig.show()

In [18]:
fig = px.density_heatmap(trips_z3_perc_df, x="Perc2", y="Perc3",text_auto=True,nbinsx=5,nbinsy=5,
                         labels={
                           "Perc2": "2 AGN [%]",
                           "Perc3": "3 AGN [%]"  
                         },
                         marginal_x="histogram",marginal_y="histogram")

fig.show()